In [43]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import SimpleImputer
# Turn interactive plotting off
plt.ioff()

# read input text and put data inside a data frame
data = pd.read_csv("../data/base_prospect_clean.csv",encoding="ISO-8859-1")
# prospect =  pd.DataFrame(prospect)
data.head
data.dtypes

code_cr             object
dept                 int64
effectif             int64
ca_total_FL          int64
ca_export_FK       float64
risque              object
endettement        float64
evo_benefice       float64
ratio_benef        float64
evo_effectif       float64
evo_risque           int64
age                  int64
type_com            object
activite            object
actionnaire         object
forme_jur_simpl     object
chgt_dir             int64
rdv                  int64
dtype: object

In [44]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown="ignore")

feature_names  = data.columns.values[2:-1]
X = data[feature_names]
y = data["code_cr"]
#La liste des caisses régionales
lst_caisse=data['code_cr'].unique()


X_cat = X.select_dtypes(exclude=['float64','int64'])

X_cat.head

# Disjonction with OneHotEncoder
encoder.fit(X_cat)
X_cat = encoder.transform(X_cat).toarray()

X.dtypes


# X_num_cat = pd.concat([pd.DataFrame(X_num), pd.DataFrame(X_cat)], axis=1)




effectif             int64
ca_total_FL          int64
ca_export_FK       float64
risque              object
endettement        float64
evo_benefice       float64
ratio_benef        float64
evo_effectif       float64
evo_risque           int64
age                  int64
type_com            object
activite            object
actionnaire         object
forme_jur_simpl     object
chgt_dir             int64
dtype: object

In [45]:
from sklearn.preprocessing import StandardScaler
# 3 - Cluster
# Normalize data
scaler = StandardScaler()
# TODO
X_num = X.select_dtypes(include=['float64','int64'])

X_num_norm = scaler.fit_transform(X_num)
# print(X_norm)



In [46]:
X_num_cat = pd.concat([pd.DataFrame(X_num_norm), pd.DataFrame(X_cat)], axis=1).to_numpy()

from sklearn.cluster import KMeans


print(X.columns)
X_num_cat



# Compute R-square, i.e. V_inter/V
from R_square_clustering import r_square
from purity import purity_score

# Plot elbow graphs for KMeans using R square and purity scores
lst_k=range(2,20)
lst_rsq = []
lst_purity = []
for k in lst_k:
    est=KMeans(n_clusters=k)
    est.fit(X_num_cat)
    lst_rsq.append(r_square(X_num_cat, est.cluster_centers_,est.labels_,k))
    # TODO: complete lst_purity
    
print(lst_purity, lst_rsq)
fig = plt.figure()
plt.plot(lst_k, lst_rsq, 'bx-')
plt.xlabel('k')
plt.ylabel('RSQ')
plt.title('The Elbow Method showing the optimal k')
plt.savefig('fig/k-means_elbow_method')
plt.close()




Index(['effectif', 'ca_total_FL', 'ca_export_FK', 'risque', 'endettement',
       'evo_benefice', 'ratio_benef', 'evo_effectif', 'evo_risque', 'age',
       'type_com', 'activite', 'actionnaire', 'forme_jur_simpl', 'chgt_dir'],
      dtype='object')
[] [0.0767573765577459, 0.14167604012758722, 0.20452221865708542, 0.266670360015165, 0.31925027916195003, 0.3554423974924842, 0.3895667246345714, 0.424090311768097, 0.43818303798177105, 0.45806373677741785, 0.49463013856852595, 0.5062476099639682, 0.5134274782691077, 0.5218605481723363, 0.5408611594123002, 0.5531611767294333, 0.5664597899037577, 0.573400971125688]


In [ ]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(
            data,
            estimator[-1].labels_,
            metric="euclidean",
            sample_size=300,
        )
    ]

    # Show the results
    formatter_result = (
        "{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}"
    )
    print(formatter_result.format(*results))

In [47]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

print(82 * "_")
print("init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette")

kmeans = KMeans(init="k-means++", n_clusters=8, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X_num_cat)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# from sklearn.datasets import make_blobs
# from sklearn.cluster import KMeans
# from sklearn.metrics import silhouette_samples, silhouette_score

# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import numpy as np
# import matplotlib.style as style

# X = X_cat

# range_n_clusters = [4, 5, 6,7,8]
# silhouette_avg_n_clusters = []

# for n_clusters in range_n_clusters:
#     # Create a subplot with 1 row and 2 columns
#     fig, (ax1, ax2) = plt.subplots(1, 2)
#     fig.set_size_inches(18, 7)

#     # The 1st subplot is the silhouette plot
#     # The silhouette coefficient can range from -1, 1 but in this example all
#     # lie within [-0.1, 1]
#     ax1.set_xlim([-0.1, 1])
#     # The (n_clusters+1)*10 is for inserting blank space between silhouette
#     # plots of individual clusters, to demarcate them clearly.
#     ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

#     # Initialize the clusterer with n_clusters value and a random generator
#     # seed of 10 for reproducibility.
#     clusterer = KMeans(n_clusters=n_clusters, random_state=42)
#     cluster_labels = clusterer.fit_predict(X)

#     # The silhouette_score gives the average value for all the samples.
#     # This gives a perspective into the density and separation of the formed
#     # clusters
#     silhouette_avg = silhouette_score(X, cluster_labels)
#     print("For n_clusters =", n_clusters,
#           "The average silhouette_score is :", silhouette_avg)

#     silhouette_avg_n_clusters.append(silhouette_avg)
#     # Compute the silhouette scores for each sample
#     sample_silhouette_values = silhouette_samples(X, cluster_labels)

#     y_lower = 10
#     for i in range(n_clusters):
#         # Aggregate the silhouette scores for samples belonging to
#         # cluster i, and sort them
#         ith_cluster_silhouette_values = \
#             sample_silhouette_values[cluster_labels == i]

#         ith_cluster_silhouette_values.sort()

#         size_cluster_i = ith_cluster_silhouette_values.shape[0]
#         y_upper = y_lower + size_cluster_i

#         color = cm.nipy_spectral(float(i) / n_clusters)
#         ax1.fill_betweenx(np.arange(y_lower, y_upper),
#                           0, ith_cluster_silhouette_values,
#                           facecolor=color, edgecolor=color, alpha=0.7)

#         # Label the silhouette plots with their cluster numbers at the middle
#         ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

#         # Compute the new y_lower for next plot
#         y_lower = y_upper + 10  # 10 for the 0 samples

#     ax1.set_title("The silhouette plot for the various clusters.")
#     ax1.set_xlabel("The silhouette coefficient values")
#     ax1.set_ylabel("Cluster label")

#     # The vertical line for average silhouette score of all the values
#     ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

#     ax1.set_yticks([])  # Clear the yaxis labels / ticks
#     ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

#     # 2nd Plot showing the actual clusters formed
#     colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
#     ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
#                 c=colors, edgecolor='k')

#     # Labeling the clusters
#     centers = clusterer.cluster_centers_
#     # Draw white circles at cluster centers
#     ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
#                 c="white", alpha=1, s=200, edgecolor='k')

#     for i, c in enumerate(centers):
#         ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
#                     s=50, edgecolor='k')

#     ax2.set_title("The visualization of the clustered data.")
#     ax2.set_xlabel("Feature space for the 1st feature")
#     ax2.set_ylabel("Feature space for the 2nd feature")

#     plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
#                   "with n_clusters = %d" % n_clusters),
#                  fontsize=14, fontweight='bold')

# plt.show()


# style.use("fivethirtyeight")
# plt.plot(range_n_clusters, silhouette_avg_n_clusters)
# plt.xlabel("Number of Clusters (k)")
# plt.ylabel("silhouette score")
# plt.savefig('fig/k-means_elbow_method')

#plt.show()